In [1]:
from scipy.integrate import odeint
from scipy import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import bqplot as bq
from ipywidgets import interact, interactive, Layout, Box, HBox, VBox, Label
import ipywidgets as ipyw
from IPython.display import display

%matplotlib inline

YEP!!! no. of args have to be equal to number of variable

see example below!

In [16]:
# Volume Normalisations
vol_er = (3.9*0.1*0.1) # all units in um
vol_cyt = (4.0*0.5*0.5-vol_er)
vol_tot = vol_cyt + vol_er
Pt=15e-6
gamma=vol_er/vol_cyt
c=100e-9
t0_cond=[Pt,0,0,0,0]
initials=[1,1,1,1,1]
eq_value=[Pt/4,Pt/4,gamma*Pt/4,gamma*Pt/4,250e-6]

def ode(x,t,k1,k_1,k2,k_2,k3):
    k_3,k4,k_4 = 1,1,1
    x1,x2,y1,y2,ce=x
    dx1 = k4*y1/gamma-k_4*x1-k1*c**2*x1+k_1*x2
    dx2 = k1*c**2*x1-k_1*x2+k_2*y2/gamma-k2*x2
    dy1 = k2*y2-k_3*ce**2*y1-k4
    dy2 = k2*x2/gamma-k_2*y2-k3*y2+k_3*ce**2*y1
    dce = 2*k3*y2-2*k_3*ce**2*y1
    return [dx1,dx2,dy1,dy2,dce]

t=np.linspace(0,0.1,100)

def func_tbs(params, value):
    X=odeint(ode,t0_cond,t,args=tuple(params))
    return X[-1]-value

#print func_tbs([5,10],value)

sol=optimize.root(func_tbs,initials,args=(eq_value,),method='broyden1',tol=1e-3)
#plot(odeint(ode,t0_cond,t,args=tuple(sol['x'])))
#plot(odeint(ode,t0_cond,t,args=tuple([1,1,1,1,1,1,1,1])))
#show()
#plot(func_tbs())
print sol

     fun: array([-1.19189110e-01, -3.75000000e-06, -1.00000152e-01, -1.52185224e-07,
       -2.50000000e-04])
 message: 'A solution was found at the specified tolerance.'
     nit: 4
  status: 1
 success: True
       x: array([-1.25338998e+15, -3.94349150e+10, -1.05159933e+15, -1.60037636e+09,
       -2.62899433e+12])


In [20]:
class slider():
    cpalette = ['#EE224A', '#22AF4B', '#4CB5F5', '#FF5C00', 
                '#08B9A5', '#15AB00', '#881EE4', '#5C6BC0']
    
    def __init__(self, model):
        self.model = model
        self.lines = {}
        self.param = {}
        self.cb = {}
        self.colors = {}
        
        self._solve()
        self._curveColors()
        self._plot()
        
        
    def _curveColors(self):
        for i,v in enumerate(self.model.label):
            self.colors.update({v: self.cpalette[i]})
        
    def _solve(self):
        sol = odeint(self.model.dXdt, self.model.X0, self.model.T)
        
        self.sol = {}
        for i,var in enumerate(self.model.label):
            self.sol.update({var: sol[:,i]})
    
    def _getFig(self):
        for v in self.model.label:
            x_sc = bq.LinearScale()
            y_sc = bq.LinearScale()

            x_ax = bq.Axis(label='Time', scale=x_sc, tick_format='0.0f', color='black', grid_lines='solid', grid_color='#ddd')
            y_ax = bq.Axis(label=v,      scale=y_sc, tick_format='0.2f', color='black', grid_lines='solid', grid_color='#ddd', orientation='vertical')

            # Generate a line (but does not plot it)
            l = bq.Lines(x=self.model.T, y=self.sol[v], colors=[self.colors[v]],
                              scales={'x': x_sc, 'y': y_sc})
        
            self.lines.update({v: l})
          
        fig = bq.Figure(axes=[x_ax, y_ax], marks=self.lines.values(),
                       fig_margin={'top':10, 'bottom':0, 'left':60, 'right':10},
                       max_aspect_ratio=3, min_aspect_ratio=2.5)
        fig.layout.width = '100%'
        return fig
        
    def _plot(self):
        self._solve()
        fig = self._getFig()
        
        display(fig)
        
    def _updateFig(self):        
        self._solve()
        for v in self.model.label:
            self.lines[v].y = self.sol[v]
    
    ### Parameter handling
    def _paramUpdate(self, change):
        self.model.p[change['owner'].description] = change['new']
        self._updateFig()
            
    def paramSlider(self):
        for k,v in self.model.p.items():
            crude = ipyw.IntSlider(
                min = -5, max = 10,
                description=k,
                value = floor(log10(v)), 
                continuous_update = False
            )

            fine = ipyw.FloatSlider(
                min = 10**floor(log10(v)),
                max = 10**(floor(log10(v))+1),
                description=k, 
                value = v,
                continuous_update = False
            )
            
            #txt = ipyw.FloatText(value=v, description=k, layout=Layout(width='80px'))
        
            #ipyw.jslink((fine, 'value'), (txt, 'value'))
            crude.observe(self._updateRange, names='value')
            fine.observe(self._paramUpdate,  names='value')
        
            p = [crude, fine]#, txt]
            self.param.update({k: p})
        
        box_layout = Layout(display='flex-start',
                    flex_flow='row',
                    align_items='flex-start',
                    align_content='flex-start',
                    width='100%')
        
        for k, v in self.param.items():
            box = Box(children=v, layout=box_layout)
            display(box, layout=Layout(align_items='flex-start'))
            
            
    def _updateRange(self, c):
        k = c['owner'].description
        try:
            self.param[k][1].max = 10**(c['new']+1)
            self.param[k][1].min = 10**c['new']
        except:
            self.param[k][1].min = 10**c['new']
            self.param[k][1].max = 10**(c['new']+1)
        self.param[k][1].value = self.param[k][1].min
        self.param[k][1].step = 1 if c['new']>0 else 10**(c['new']-1)
        
    ### Toggle curve display
    def _toggleCurve(self, b):
        v = not self.lines[b.description].visible
        if self.lines[b.description].visible:
            self.cb[b.description].style.button_color = '#ddd'
            self.lines[b.description].visible = v
        else:
            self.cb[b.description].style.button_color = self.colors[b.description]
            self.lines[b.description].visible = v
        
        
    def _getToggleButton(self, v):
        cb = ipyw.Button(
            value = self.lines[v].visible,
            description = v,
            style = ipyw.ButtonStyle(button_color=self.colors[v]),
            layout = Layout(width='100px')
        )
        return cb
        
    def toggleButton(self):
        for v in self.model.label:
            self.cb.update({v: self._getToggleButton(v)})
            
        display(VBox(self.cb.values()))
        
        for cb in self.cb.values():
            cb.on_click(self._toggleCurve)

In [21]:
### Four state serca model class
class serca:
    
    ## Must be in same order as dXdt return
    label = ['x1', 'x2', 'y1','y1','ce']
    nVar = len(label)
    
    vol_er = (3.9*0.1*0.1) # all units in um
    vol_cyt = (4.0*0.5*0.5-vol_er)
    vol_tot = vol_cyt + vol_er
    Pt=15e-6
    gamma=vol_er/vol_cyt
    c=100e-9
    
    #eq_value=[Pt/4,Pt/4,gamma*Pt/4,gamma*Pt/4,250e-6]
    
    ## Parameters
    p = {
    'k1':  1,
    'k_1': 1,
    'k2':  1,
    'k_2': 1,
    'k3':  1,
    'k_3': 1,
    'k4':  1,
    'k_4': 1,
    }
    
    ## Get initial values for the system
    def __init__(self, X0=[Pt,0,0,0,0], T=arange(0.0, 50.0, 0.01)):
        self.T = T
        self.name = self.__class__.__name__
        if self.nVar != len(X0):
            print 'ERROR:', self.nVar, 'initial values required for:', self.name
        else: 
            self.X0 = X0
            
    def dXdt(self,X,t):
        x1,x2,y1,y2,ce=X
        for var in self.p.keys():
            exec(var+'='+str(self.p[var]))
        
        dx1 = k4*y1/self.gamma-k_4*x1-k1*self.c**2*x1+k_1*x2
        dx2 = k1*self.c**2*x1-k_1*x2+k_2*y2/self.gamma-k2*x2
        dy1 = k2*y2-k_3*ce**2*y1-k4
        dy2 = k2*x2/self.gamma-k_2*y2-k3*y2+k_3*ce**2*y1
        dce = 2*k3*y2-2*k_3*ce**2*y1
        return [dx1,dx2,dy1,dy2,dce]

In [22]:
#X0 = [0.3, 0.2, 0.1]
#tmin, tmax, dt = 0, 30, 0.01
#T = arange(tmin,tmax,dt)
m = slider(model=serca())

m.paramSlider()
m.toggleButton()

RmlndXJlKGF4ZXM9W0F4aXMoY29sb3I9J2JsYWNrJywgZ3JpZF9jb2xvcj0nI2RkZCcsIGxhYmVsPXUnVGltZScsIHNjYWxlPUxpbmVhclNjYWxlKCksIHRpY2tfZm9ybWF0PXUnMC4wZicpLCDigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUnazMnLCBtYXg9MTAsIG1pbj0tNSksIEZsb2F0U2xpZGVyKHZhbHXigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUnazInLCBtYXg9MTAsIG1pbj0tNSksIEZsb2F0U2xpZGVyKHZhbHXigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUnazEnLCBtYXg9MTAsIG1pbj0tNSksIEZsb2F0U2xpZGVyKHZhbHXigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUnazQnLCBtYXg9MTAsIG1pbj0tNSksIEZsb2F0U2xpZGVyKHZhbHXigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna18xJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcih2YWzigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna18zJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcih2YWzigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna18yJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcih2YWzigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna180JywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcih2YWzigKY=


VkJveChjaGlsZHJlbj0oQnV0dG9uKGRlc2NyaXB0aW9uPXUneDInLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwcHgnKSwgc3R5bGU9QnV0dG9uU3R5bGUoYnV0dG9uX2NvbG9yPScjMjJBRjTigKY=
